# LLM test

In [ ]:
# py
"""
Module for generating contextual questions about captured images using a local LLM.
Works with the User_Detection module to provide personalized image understanding.
"""

import logging
import os
from typing import List, Dict, Any
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

logger = logging.getLogger(__name__)

class ImageContextGenerator:
    def __init__(self, model_path: str = "local_models/llama3-8b"):
        """
        Initialize the image context question generator with a local LLM.
        
        Parameters:
            model_path (str): Path to the local model files
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Loading LLM model from {model_path} on {self.device}")
        
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                low_cpu_mem_usage=True,
                device_map="auto"
            )
            logger.info("LLM model loaded successfully")
        except Exception as e:
            logger.error(f"Failed to load LLM model: {e}")
            raise
    
    def generate_questions(self):
        prompt = f"""
        Tell me a Joke
        """
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs["input_ids"],
                max_new_tokens=150,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract just the generated questions part
        questions_text = response.split("Questions:")[-1].strip()
        
        # Parse into separate questions (assuming numbered or line-separated format)
        questions = [q.strip() for q in questions_text.split("\n") if q.strip()]
        
        logger.info(f"Generated {len(questions)} contextual questions")
        return questions


In [ ]:
import sys
import os

# Add parent directory to path to import modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the module
from Image_Understanding.context_generator import ImageContextGenerator

# Initialize the generator with a small test model
context_gen = ImageContextGenerator(model_path="TheBloke/Phi-3-mini-4k-instruct-GGUF")


answer = context_gen.get_answer()

print("\nGenerated answer:")
print(answer)